In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [10]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [11]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [12]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


In [13]:
# Set up some global variables
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /device:GPU:0


In [14]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):

        # Compute the loss like we did in Part II
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [20]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        num_classes = 10
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv1 = tf.keras.layers.Conv2D(32, (3,3), strides = 1, padding='same'
                                            ,activation = tf.nn.relu)
        self.pool1 = tf.keras.layers.MaxPool2D((2,2))
        self.norm1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(64, (3,3), strides = 1, padding='same',
                                            activation = tf.nn.relu)
        self.pool2 = tf.keras.layers.MaxPool2D((2,2))
        self.norm2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(256, (3,3), strides = 1, padding='same',
                                      activation = tf.nn.relu)
        self.pool3 = tf.keras.layers.MaxPool2D((2,2))
        self.norm3 = tf.keras.layers.BatchNormalization()
        self.conv4 = tf.keras.layers.Conv2D(512, (3,3), strides = 1, padding='same',
                                      activation = tf.nn.relu)
        self.pool4 = tf.keras.layers.MaxPool2D((2,2))
        self.norm4 = tf.keras.layers.BatchNormalization()
        self.fc1 = tf.keras.layers.Dense(128, 
                                       activation = tf.nn.relu)
        self.drop1 = tf.keras.layers.Dropout(0.3)
        self.normf1 = tf.keras.layers.BatchNormalization()
        self.fc2 = tf.keras.layers.Dense(256, 
                                       activation = tf.nn.relu)
        self.drop2 = tf.keras.layers.Dropout(0.3)
        self.normf2 = tf.keras.layers.BatchNormalization()
        self.fc3 = tf.keras.layers.Dense(512, 
                       kernel_initializer = initializer,bias_initializer = initializer)
        self.fc4 = tf.keras.layers.Dense(256, 
                      kernel_initializer = initializer,bias_initializer = initializer)
                              
        self.final = tf.keras.layers.Dense(num_classes, use_bias = True,
                      kernel_initializer = initializer,bias_initializer = initializer)
                                     
       


        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)
        x = self.pool1(x)
        x = self.norm1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.norm2(x)        
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.norm3(x)
        x = self.conv4(x)
        x = self.pool4(x)
        x = self.norm4(x)         
        x = tf.keras.layers.Flatten()(x)
        x = self.fc1(x)
        x = self.drop1(x)
        x = self.normf1(x)
        x = self.fc2(x)
        x = self.drop2(x)
        x = self.normf2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.final(x)
        
       

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        
        return x


In [18]:
device = '/device:GPU:0'
learning_rate = 1e-3

def model_init_fn():
    model = None
    model = CustomConvNet()
    return model

def optimizer_init_fn():
    optimizer = None
    optimizer = tf.keras.optimizers.Adam(learning_rate) 
    return optimizer
    
print_every = 700
num_epochs = 10
train_part34(model_init_fn, optimizer_init_fn, num_epochs)

Iteration 0, Epoch 1, Loss: 2.3030850887298584, Accuracy: 7.8125, Val Loss: 2.339470863342285, Val Accuracy: 12.300000190734863
Iteration 700, Epoch 1, Loss: 1.3052279949188232, Accuracy: 52.2913703918457, Val Loss: 0.9409611821174622, Val Accuracy: 67.0999984741211
Iteration 1400, Epoch 2, Loss: 0.7719188332557678, Accuracy: 72.9502944946289, Val Loss: 0.7245126962661743, Val Accuracy: 75.0
Iteration 2100, Epoch 3, Loss: 0.5401200652122498, Accuracy: 81.09622192382812, Val Loss: 0.7796425819396973, Val Accuracy: 74.29999542236328
Iteration 2800, Epoch 4, Loss: 0.3796386420726776, Accuracy: 86.86319732666016, Val Loss: 0.9057743549346924, Val Accuracy: 75.19999694824219
Iteration 3500, Epoch 5, Loss: 0.2777491807937622, Accuracy: 90.27460479736328, Val Loss: 0.863075852394104, Val Accuracy: 74.5
Iteration 4200, Epoch 6, Loss: 0.20162758231163025, Accuracy: 92.90347290039062, Val Loss: 0.9895452260971069, Val Accuracy: 75.0999984741211
Iteration 4900, Epoch 7, Loss: 0.1633986234664917, 

## Describe what you did 

In the cell below you should write an explanation of what you did, any additional features that you implemented, and/or any graphs that you made in the process of training and evaluating your network.

Network Architecture: 

(conv - maxpooling - batchnorm ) * 4 - (fc - maxpooling - batchnorm) * 2 - fc * 3

Regularization: Dropout

Optimizer: Adam with learning rate 1e-3.